In [102]:
# %%writefile Factor_Cacher.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd
import numpy as np

import tools.Sample_Tools as smpl
import base.JuUnits as ju

__cache_dir__ = ju.get_root_path()+'/cache.feather/'
__cache_type__ = '.feather'

# df = smpl.get_sample_by_zs(name='沪深300', end='2021-11-28', gap=2500,  only_main=True, filter_st=True).data
# df.to_parquet(__cache_dir__+'test.parquet')

def file_path(file_name):
    return __cache_dir__ + file_name + __cache_type__

def is_cache_exist(name):
    return os.path.exists(file_path(name))

def save_cache(name, data):
    df.reset_index().to_feather(file_path(name))
    
def load_cache(name, to_series=False):
    assert is_cache_exist(name),'cache not exist, create first'
    df = pd.read_feather(file_path(name)).set_index(['date','code'])
    if to_series:
        df =  df.squeeze()
    return df

def load_cache_adv(name, start, end, to_series=False):
    assert is_cache_exist(name),'cache not exist, create first'
        
    st = pd.Timestamp(start) 
    en = pd.Timestamp(end) 

    df = pd.read_feather(file_path(name)).set_index(['date','code'])[st:en]
    date_idx = df.index.get_level_values(0).unique()
    if not (st >= date_idx.min() and en <= date_idx.max()):
        info = 'date range out of cache[{},{}]'.format(
            date_idx.min().strftime('%Y-%m-%d %H:%M:%S'), 
            date_idx.max().strftime('%Y-%m-%d %H:%M:%S')
        )
        raise BaseException(info)
    
    if to_series:
        df =  df.squeeze()
    return df

In [31]:
%%time
df.reset_index().to_feather(__cache_dir__+'test.feather')

Wall time: 71.8 ms


In [107]:
# df['open'].reset_index().to_feather(__cache_dir__+'test_s.feather')
# pd.read_feather(__cache_dir__+'test_s.feather').set_index(['date','code']).squeeze()
# df.squeeze()
